In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import copy

In [2]:
df=pd.read_csv('play_tennis.csv')
df

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [3]:
df1=df

In [4]:
# df1['outlook']=df['outlook'].replace({'Sunny':0,'Overcast':1,'Rain':2})
# df1

In [5]:
# df1['temp']=df1['temp'].replace({'Hot':0,'Mild':1,'Cool':2})
# df1['humidity']=df['humidity'].replace({'Normal':0,'High':1})
# df1['wind']=df['wind'].replace({'Weak':0,'Strong':1})
# df1['play']=df['play'].replace({'No':0,'Yes':1})
# df1

In [6]:
df1.drop(['day'],axis=1,inplace=True)
df1

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [7]:
df1.outlook.value_counts()

Rain        5
Sunny       5
Overcast    4
Name: outlook, dtype: int64

In [8]:
x=df1['play']
x

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: play, dtype: object

In [9]:
np.array(x.value_counts())

array([9, 5], dtype=int64)

In [10]:
class DecisionTree:
    def __init__(self):
        self.value=""
        self.nodes={}
    def __str__(self):
        return self.value+f' {self.nodes}'

In [11]:
def calc_entropy(column):
    total=len(column)
    category=np.array(column.value_counts())
    entropy=0
    for i in category:
        entropy-=(i/total)*np.log2(i/total)
    return entropy
    

In [12]:
def calc_information_gain(dataframe,attribute,target_entropy,target):
    column=dataframe[attribute]
    n=len(column)
    values=column.value_counts().tolist()
    classes=column.value_counts().keys().tolist()
    entropy_of_class=[]
    for i in classes:
        col=dataframe[column==i][target]
        entropy_of_class.append(calc_entropy(col))
    ig=target_entropy
    category_entropy={}
    for i in range(len(values)):
        ig-=(entropy_of_class[i]*(values[i]/n))
        category_entropy[classes[i]]=entropy_of_class[i]
    return ig,category_entropy

In [13]:
def build_decisionTree(tree,dataframe,target,target_entropy):
    attributes=np.array(dataframe.columns[:-1])
    max_ig=0;
    child_node={}
    parent_node=""
    
    for attribute in attributes:
        information_gain,children=calc_information_gain(dataframe,attribute,target_entropy,target)
        if(information_gain>max_ig):
            max_ig=information_gain
            child_node=children
            parent_node=attribute

    tree.value=parent_node
    for i in list(child_node.keys()):
        child_tree=DecisionTree()
        temp_df=dataframe[dataframe[parent_node]==i]
        new_df=temp_df.drop([parent_node],axis=1)
        if(child_node[i]==0):
            tree.nodes[i]=temp_df[target].unique()[0]
        else:
            tree.nodes[i]=child_tree
            build_decisionTree(child_tree,new_df,target,child_node[i])

In [14]:
tree=DecisionTree()
target='play'
target_entropy=calc_entropy(df1[target])
build_decisionTree(tree,df1,target,target_entropy)

In [15]:
print(tree)

outlook {'Rain': <__main__.DecisionTree object at 0x00000283000B5C40>, 'Sunny': <__main__.DecisionTree object at 0x000002830009CB50>, 'Overcast': 'Yes'}


In [16]:
print(tree.nodes['Sunny'])

humidity {'High': 'No', 'Normal': 'Yes'}


In [17]:
print(tree.nodes['Rain'])

wind {'Weak': 'Yes', 'Strong': 'No'}
